## Model Export Framework for PLC Deployment  

This module is developed as part of the research project:  

**_“Towards AI-Based Anomaly Detection at the Edge:  
Evaluating Real-Time Cyber Defense in Programmable Logic Controllers”_**  

The implementation establishes a deterministic and reproducible framework for deploying LSTM-based anomaly detection models on Programmable Logic Controllers (PLCs) in research and educational contexts.  

It enables consistent model loading from PyTorch checkpoints through a parameter-aligned interface that supports both full model serialization and `state_dict` checkpoints.  
All LSTM parameters — including input and recurrent weights, biases, and fully connected layer coefficients — are extracted per gate (`i, f, g, o`) and converted into PLC-native data structures.  

Two standardized export formats are produced:  
- **Siemens DATA_BLOCK (`.db`)** – for Siemens© PLC 'db' variable initialization.  
- **Structured Text (`.st`)** – for for Beckhoff© PLC 'st' variable initialization.  

The framework includes a user-confirmed overwrite mechanism for safe file handling and employs a hyperparameter-based naming convention to ensure traceability between the original neural network and its PLC-deployable form.  

This mechanism constitutes a core technical component of the above-mentioned research, supporting AI-based anomaly detection at the industrial edge and enabling real-time, resource-efficient, and auditable cyber defense in critical automation systems.  

---

## User Guide — Configuration and Execution  

This module automatically locates and exports the correct trained LSTM model based on a minimal configuration defined at the beginning of the script.  
The user only needs to configure a few parameters; all paths and exports are handled automatically.

---

### Step 1: Configure Model Parameters  

Before running the script, set these constants according to the trained model you want to export:

```python
MODEL_NAME = "LSTM_SWaT"
MODEL_VERSION = "v1"
HIDDEN_SIZE = 8
SEQUENCE_LENGTH = 10
BEST_EPOCH = 2
```

---

### Step 2: Run the Script  

Run the notebook or script manually (no need for “Run All”).  
Confirm overwrite if prompted.  
No additional user action is required — the export files will be created automatically.  

---

### Step 3: Check the Output  


After completion, verify that the following files exist in the model folder:
The exact filenames depend on the user-defined configuration parameters
(e.g., model name, hidden size, input window, version, and epoch).

Example output files:
- `LSTM_SWaT_HS8_IW10_v1_Epoch_02.db`
- `LSTM_SWaT_HS8_IW10_v1_Epoch_02.st`

These represent the PLC-ready exports of your trained model.  

---

## Legal Notice  

- **Author:** Anonymous  
- **Website:** ---  
- **Contact:** Anonymous  
- **License:** Creative Commons BY-NC 4.0  
- **Version:** v1.0.1  
- **Copyright:** © 2025–2026  

This software is intended exclusively for **educational and research purposes**, or other **non-commercial applications**.  
Use in **industrial production environments** is strictly prohibited.  
The **author must be credited** in all derivative or redistributive works.  
**Commercial use** requires **explicit written permission** from the author.  
